**Installs**

In [1]:
!pip install flask

In [2]:
!pip install flask-ngrok

In [3]:
!pip install pydub speechbrain datasets soundfile
!pip install git+https://github.com/huggingface/transformers.git
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 22.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 r

---

In [4]:
from datasets import load_dataset, Audio
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from speechbrain.pretrained import EncoderClassifier
import torch
import os
from flask import Flask, request, render_template_string, jsonify
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from speechbrain.pretrained import EncoderClassifier
import torch
import os
import io
from pydub import AudioSegment
from flask import send_file
from IPython.display import Audio

<ipython-input-4-3a2abf9e22a0>:3: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


In [5]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://fkrkg0t5xu-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [6]:
from datasets import load_dataset, Audio


# Load the dataset in streaming mode
dataset = load_dataset(
    "facebook/voxpopuli", "en", streaming=True, trust_remote_code=True
)

dataset = {
    'train': dataset["train"],
    'validation': dataset["validation"],
    'test': dataset["test"]
}
for split in dataset.values():
    split.cast_column("audio", Audio(sampling_rate=16000))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

voxpopuli.py:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

---

**make generated_audio.wav inside sample_data to work or just change path**

In [11]:
# @title Default title text
app = Flask(__name__)

# Load your models and processors
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("Magdo66/speecht5_tts_voxpopuli_en")
tokenizer = processor.tokenizer

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name)
)

def prepare_dataset(example):
    # load the audio data; if necessary, this resamples the audio to 16kHz
    audio = example["audio"]

    # feature extraction and tokenization
    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

def generate_speech(text):
    inputs = processor(text=text, return_tensors="pt")
    example = prepare_dataset(next(iter(dataset["test"])))

    speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

    spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)

    with torch.no_grad():
        speech = vocoder(spectrogram)

    audio_data = speech.squeeze().cpu().numpy()
    if audio_data.size == 0:
        raise ValueError("Generated audio data is empty")

    wav_buffer = io.BytesIO()
    from scipy.io.wavfile import write
    write(wav_buffer, 16000, audio_data)  # 16000 Hz is the sampling rate for SpeechT5
    wav_buffer.seek(0)  # Ensure the pointer is at the beginning of the BytesIO buffer

    return wav_buffer



# HTML form as a string
html_form = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Text to Speech</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f9;
            color: #333;
            margin: 0;
            padding: 20px;
            display: flex;
            flex-direction: column;
            align-items: center;
        }

        h1 {
            color: #4CAF50;
        }

        form {
            background: #fff;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            width: 100%;
            max-width: 500px;
        }

        label {
            font-weight: bold;
        }

        textarea {
            width: 100%;
            border: 1px solid #ddd;
            border-radius: 4px;
            padding: 10px;
            font-size: 16px;
            resize: none;
            margin-top: 5px;
        }

        button {
            background: #4CAF50;
            color: white;
            border: none;
            padding: 10px 20px;
            font-size: 16px;
            border-radius: 4px;
            cursor: pointer;
            margin-top: 10px;
        }

        button:hover {
            background: #45a049;
        }

        audio {
            margin-top: 20px;
            width: 100%;
            max-width: 500px;
        }

        .error {
            color: red;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <h1>Text to Speech</h1>
    <form id="tts-form">
        <label for="text">Enter Text:</label><br>
        <textarea id="text" name="text" rows="4" cols="50" placeholder="Type something..." required></textarea><br><br>
        <button type="submit">Generate Speech</button>
    </form>
    <audio id="audio-player" controls>
        <source id="audio-source" src="" type="audio/wav">
        Your browser does not support the audio element.
    </audio>

    <script>
        document.getElementById('tts-form').addEventListener('submit', async function(event) {
            event.preventDefault(); // Prevent page refresh

            const text = document.getElementById('text').value;
            if (!text.trim()) {
                alert("Please enter some text.");
                return;
            }

            try {
                const response = await fetch('/', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/x-www-form-urlencoded' },
                    body: new URLSearchParams({ text: text })
                });

                if (!response.ok) {
                    const error = await response.text();
                    alert(`Error: ${error}`);
                    return;
                }

                const blob = await response.blob(); // Receive the audio file
                const audioUrl = URL.createObjectURL(blob); // Create a URL for the audio file

                const audioSource = document.getElementById('audio-source');
                audioSource.src = audioUrl; // Update the audio source
                const audioPlayer = document.getElementById('audio-player');
                audioPlayer.load(); // Reload the audio player
                audioPlayer.play(); // Play the new audio
            } catch (error) {
                console.error('Error:', error);
                alert('An error occurred while generating the audio.');
            }
        });
    </script>
</body>
</html>


'''

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        text = request.form.get('text', '')
        if not text:
            return "Text is required", 400

        # Generate audio file
        wav_buffer = generate_speech(text)
        wav_buffer.seek(0)

        return send_file(wav_buffer, mimetype="audio/wav", as_attachment=False)

    return render_template_string(html_form)

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)  # use_reloader=False to pr

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Dec/2024 21:02:02] "GET /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2024 21:02:02] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2024 21:02:37] "POST /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2024 21:02:38] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2024 21:03:05] "POST /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2024 21:03:38] "POST /?authuser=0 HTTP/1.1" 200 -


---